In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import re
from src.DocumentReader import read_n_documents
from src.shared import process

##### The whole dataset has 2.250.223 documents

In [2]:
docs = read_n_documents(0,10)
processed_documents = process(docs)
processed_documents

,title,version,abstract,authors_parsed,category_groups
0,Calculation of prompt diphoton production cros...,2007,A fully differential calculation in perturba...,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...",[hep-ph]
1,Sparsity-certifying Graph Decompositions,2008,"We describe a new algorithm, the $(k,\ell)$-...","[[Streinu, Ileana, ], [Theran, Louis, ]]","[math, cs]"
2,The evolution of the Earth-Moon system based o...,2008,The evolution of Earth-Moon system is descri...,"[[Pan, Hongjun, ]]",[physics]
3,A determinant of Stirling cycle numbers counts...,2007,We show that a determinant of Stirling cycle...,"[[Callan, David, ]]",[math]
4,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,2007,In this paper we show how to compute the $\L...,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]","[math, math]"
5,Bosonic characters of atomic Cooper pairs acro...,2007,We study the two-particle wave function of p...,"[[Pong, Y. H., ], [Law, C. K., ]]",[cond-mat]
6,Polymer Quantum Mechanics and its Continuum Limit,2007,A rather non-standard quantum representation...,"[[Corichi, Alejandro, ], [Vukasinac, Tatjana, ...",[gr-qc]
7,Numerical solution of shock and ramp compressi...,2008,A general formulation was developed to repre...,"[[Swift, Damian C., ]]",[cond-mat]
8,"The Spitzer c2d Survey of Large, Nearby, Inste...",2007,We discuss the results from the combined IRA...,"[[Harvey, Paul, ], [Merin, Bruno, ], [Huard, T...",[astro-ph]
9,"Partial cubes: structures, characterizations, ...",2007,Partial cubes are isometric subgraphs of hyp...,"[[Ovchinnikov, Sergei, ]]",[math]


##### Map Categories like ["Physics", "Maths"] to indexes like [0,1]

In [19]:
def map_categories_to_indexes(categories):
    unique_categories = []
    all_indexes = []
    for i in categories:
        document_indexes = []
        for category in i:
            if category not in unique_categories:
                unique_categories.append(category)
                document_indexes.append(len(unique_categories)-1)
            else:
                if unique_categories.index(category) not in document_indexes:
                    document_indexes.append(unique_categories.index(category))
        all_indexes.append(document_indexes)
    return all_indexes, unique_categories


In [20]:
all_indexes, labels = map_categories_to_indexes(processed_documents['category_groups'].values)
# print(all_indexes)

##### Map indexes like [[0,1],[1,2]] to [[1,1,0],[0,1,1]]

In [21]:
from sklearn.preprocessing import MultiLabelBinarizer
import scipy.sparse as sp
mlb = MultiLabelBinarizer()

# y = sp.csr_array(mlb.fit_transform(all_indexes))
y = mlb.fit_transform(all_indexes)

In [22]:
y

array([[1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0]])

In [23]:
y.shape

(10, 7)

In [24]:
labels

['hep-ph', 'math', 'cs', 'physics', 'cond-mat', 'gr-qc', 'astro-ph']

<BarContainer object of 7 artists>

In [25]:
label_counts = y.sum(axis=0)
plt.bar(np.linspace(1,18,len(label_counts)), label_counts)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack

#What does the regular expression r'w{1,} do?
count_vect = CountVectorizer(analyzer='word')
# count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')

#"fit" is actually building the vocabulary
count_vect.fit(processed_documents['abstract'].values)

#the transformation converts the text to bag-of-words
X = count_vect.transform(processed_documents['abstract'].values)  #these are sparse BOW matrices for Q1

##### Class counts

### Split data into training and test sets

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, shuffle = True, random_state = 42)

In [28]:
print(y.shape)
print(y_train.shape)
print(y_test.shape)

(10, 7)
(7, 7)
(3, 7)


In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier

tree_model = DecisionTreeClassifier()
multilabel_model = MultiOutputClassifier(tree_model).fit(X_train,y_train)
predictions = multilabel_model.predict(X_test)

In [30]:
from sklearn.metrics import f1_score, classification_report, accuracy_score

f1_score(predictions, y_test, average="macro")

c:\Users\staso\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.0

In [31]:
f1_score(predictions[:,0], y_test[:,0], average="macro")


1.0

In [32]:
scores = []
print("F1 scores for each category")
print()
for i in range(y_test.shape[1]):
    score = f1_score(predictions[:,i], y_test[:,i], average="macro")
    score = round(score,2)*100
    print(labels[i]+":", int(score), "% with count:", label_counts[i])
    scores.append(score)

F1 scores for each category

hep-ph: 100 % with count: 1
math: 40 % with count: 4
cs: 40 % with count: 1
physics: 100 % with count: 1
cond-mat: 40 % with count: 2
gr-qc: 100 % with count: 1
astro-ph: 40 % with count: 1


In [33]:
print("Weighted F1-scores",np.average(np.array(scores), weights=label_counts))
print("Macro F1-scores", np.average(np.array(scores)))

Weighted F1-scores 56.36363636363637
Macro F1-scores 65.71428571428571


In [36]:
fig, ax1 = plt.subplots()

ax1.set_title("12.000 dataset model performance")
color = 'tab:blue'
ax1.set_xlabel('Category index')
ax1.set_ylabel('Category count [n]', color=color)
ax1.bar(np.linspace(1,18,len(label_counts)), label_counts, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('Accuracy [%]', color=color)  # we already handled the x-label with ax1
ax2.scatter(np.linspace(1,18,len(label_counts)), scores, c="red")
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()
